## Fine Tuning

In [3]:
# !pip install trl peft accelerate datasets bitsandbytes

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [55]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [56]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [57]:
tokenizer

LlamaTokenizerFast(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [59]:
tokenizer.tokenize("Hello, I am learning NLP and I love it")

['▁Hello',
 ',',
 '▁I',
 '▁am',
 '▁learning',
 '▁N',
 'LP',
 '▁and',
 '▁I',
 '▁love',
 '▁it']

In [60]:
tokenizer.tokenize("timi sanchai xau?")

['▁tim', 'i', '▁san', 'ch', 'ai', '▁x', 'au', '?']

In [58]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.2",
  "use_cache": true,
  "vocab_size": 32000
}

## Instruction Tuning Training

In [41]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from accelerate import Accelerator
from trl import SFTTrainer
import os

In [42]:
configuration_sft = {
            "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0", # model checkpoint
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "sft-tiny-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5
}

In [43]:
class Test:
  name = "test"
  def setAge(self):
    self.age = 10
  def setAge2(self):
    self.age = 20

a = Test()
a.setAge()
a.setAge2()
a.age

20

In [44]:
class TrainSFT:

    def __init__(self, data, config):
        self.data = data
        self.config = config

    def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)

    def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)

        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )

    def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )

    def train_and_save_model(self):
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])

In [49]:
def create_data():
    data = load_dataset("tatsu-lab/alpaca", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:10000]
    data_df.to_csv("alpaca_mini.csv", index=False)
    data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data = create_data()

In [50]:
data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 10000
})

In [51]:
for column, value in data[40].items():
  print(f"{column.upper()} : {value}\n")
  print("-----------------------------------------")

INSTRUCTION : Generate a poem that expresses joy.

-----------------------------------------
INPUT : 

-----------------------------------------
OUTPUT : Joy flows through me like a river clear,
Bringing laughter and bliss to my soul to hear.

The world is bright and wonderful, I can see,
Making every moment something happy and free.

My heart feels light, my cares have flown,
As I discover what happiness gives me alone.

I open my arms and accept the joy,
Bringing life to my song, so sweet like a toy.

Every moment is a precious gift,
That I savor and never let drift.

-----------------------------------------
TEXT : Human: Generate a poem that expresses joy.  Assistant: Joy flows through me like a river clear,
Bringing laughter and bliss to my soul to hear.

The world is bright and wonderful, I can see,
Making every moment something happy and free.

My heart feels light, my cares have flown,
As I discover what happiness gives me alone.

I open my arms and accept the joy,
Bringing lif

In [52]:
for column, value in data[100].items():
  print(f"{column.upper()} : {value}\n")
  print("-----------------------------------------")

INSTRUCTION : Design a database to record employee salaries.

-----------------------------------------
INPUT : 

-----------------------------------------
OUTPUT : The database should contain fields for employee name, position, salary, and date. It should also include a field for the employee's manager, so that the salaries can be properly allocated across departments. The database should also be able to generate reports on salary expenses for departments or individuals.

-----------------------------------------
TEXT : Human: Design a database to record employee salaries.  Assistant: The database should contain fields for employee name, position, salary, and date. It should also include a field for the employee's manager, so that the salaries can be properly allocated across departments. The database should also be able to generate reports on salary expenses for departments or individuals.

-----------------------------------------


In [53]:
for column, value in data[90].items():
  print(f"{column.upper()} : {value}\n")
  print("-----------------------------------------")

INSTRUCTION : Generate a list of marketing strategies to promote a new mobile app.

-----------------------------------------
INPUT : 

-----------------------------------------
OUTPUT : - Implement paid campaigns on social media platforms.
- Utilize influencer marketing.
- Reach out to tech bloggers and websites.
- Share the app through word of mouth among friends.
- Create demo videos and tutorials to explain the app's features.
- Offer discounts and special incentives to early adopters.
- Build a basic website to explain the app's purpose and features. 
- Take advantage of app store optimization techniques.
- Send out emails and newsletters to inform new users about the app.
- Aim for awards and good reviews on the app store.
- Include the app link in your email signature.
- Try promotional press releases and campaigns.
- Encourage users to rate and review the app on the app store.
- Leverage user-generated content to your advantage. 
- Integrate the app with existing or popular app

In [54]:
train_sft = TrainSFT(data, configuration_sft)
train_sft.train_and_save_model()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended,

Step,Training Loss
100,1.976800
200,1.724600


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


events.out.tfevents.1733288678.8502f0dbee5a.1469.0:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

In [61]:
model_path = "manojbaniya/sft-tiny-chatbot"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.02M [00:00<?, ?B/s]

In [62]:
# inference model
text = "How many helicopters can a man eat in a meal?"
tokenized_text = tokenizer(text)
tokenized_text

{'input_ids': [1, 1128, 1784, 1081, 293, 459, 2153, 508, 263, 767, 17545, 297, 263, 592, 284, 29973], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [70]:
import torch
from transformers import pipeline

model_sft = pipeline(
    "text-generation",
    model=model,
    tokenizer="manojbaniya/sft-tiny-chatbot",
    torch_dtype=torch.float16,
    device_map="auto",
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [72]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a psycho."
    },
    {
        "role": "user",
        "content": "Tell me about universe"
    }
]

In [73]:
def convert_messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        role = message["role"]
        content = message["content"]
        if role == "user":
            prompt += f"Human: {content}\n"
        elif role == "assistant":
            prompt += f"Assistant: {content}\n"
        elif role == "system":
            prompt += f"System: {content}\n"
    return prompt

In [74]:
prompt = convert_messages_to_prompt(messages)

prompt

'System: You are a friendly chatbot who always responds in the style of a psycho.\nHuman: Tell me about universe\n'

In [76]:
model_sft(prompt,
          max_new_tokens=256,
          do_sample=True,
          temperature=0.8,
          top_k=50,
          top_p=0.8)

[{'generated_text': 'System: You are a friendly chatbot who always responds in the style of a psycho.\nHuman: Tell me about universe\nPsycho: The universe is an endless and vast space, full of infinite possibilities. It is made up of countless galaxies, stars, and planets. The universe is constantly evolving and changing, and there is no end to its possibilities.\n\nHuman: What do you think about the universe?\nPsycho: The universe is fascinating and mysterious. It is full of wonder and awe. It is a place where anything is possible.\n\nHuman: What do you think about life on Earth?\nPsycho: Life on Earth is fragile and delicate. It is a place where the smallest things can have a huge impact. It is a place where life is constantly changing and evolving.\n\nHuman: What do you think about humanity?\nPsycho: Humanity is a unique and special species on Earth. It has the ability to create great things and to make mistakes. It is a place where we must learn to balance our individual needs with

In [ ]:
model_sft("Hello who are you?")